# 개요

멜론, 벅스, 지니의 시대별 히트곡 가사로 분류한다.

## 3조
* 김성모, 이성연, 이현중, 전민정

## 진행 순서

1. 크롤링
2. 전처리

## 타임테이블

https://timetr.ee/s/i48rvoYnhUOfgLkrswFhKFijqdVi8vVA

## 레이블
* 희망
* 돈
* 사랑 - 연애, 이별
* 효 - 어머니, 아버지 ...
* 우정 - 친구
* 남녀평등 - 걸크러쉬
* 욕설
* 자연
* 후회
* 추억
* SF
* 음식
* 자신감
* 즐거움


# 크롤링

## 멜론

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [ ]:
years, singers, songs, lyrics = [],[],[],[]
start_year = 1964
final_year = 2022
for i in tqdm(range(start_year, final_year)):

  #각 년도별 top 100 노래를 담은 url 추출
  url = 'https://www.melon.com/chart/age/list.htm?idx=1&chartType=YE&chartGenre=KPOP&chartDate='+str(i)+'&moved=Y'

  headers = {'Referer':'https://www.melon.com/song/detail.htm?songId=33239419',
            'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36'}

  resp = requests.get(url, headers = headers)
  soup = BeautifulSoup(resp.content, 'lxml')
  id_tag = soup.select('div.wrap button')
  data = []

  for j in range(len(id_tag)):
    try:
      #각 노래의 고유번호 data 리스트에 저장해주기
      id_tag[j]['data-song-no']
      data.append(id_tag[j]['data-song-no'])
    except:
      pass

  for dat in data:
    link = 'https://www.melon.com/song/detail.htm?songId=' + dat

    headers = {'Referer':'https://www.melon.com/song/detail.htm?songId=' + dat,
              'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36'}


    resp = requests.get(link, headers = headers)
    soup = BeautifulSoup(resp.content, 'lxml')
    lyric_tag = soup.select('div.wrap_lyric')
    song_tag = soup.select('div.song_name')
    try:
      singer_tag = soup.select('div.artist a')
      singer_tag[1]
    except:
      singer_tag = soup.select('div.artist')
    raw = lyric_tag[0].text
    lyric = raw[:-6].strip()
    song = song_tag[0].text.strip()[2:].strip()
    singer = singer_tag[0].text.strip()
    lyrics.append(lyric)
    songs.append(song)
    singers.append(singer)
    years.append(i)


In [ ]:
# csv 로 정리하여 출력
dict_song = {'years':years,
             'singers':singers,
             'songs':songs,
             'lyrics':lyrics}

df = pd.DataFrame(dict_song)
df.to_csv('songs_ranking.csv', index=False)

## 벅스

- 연도별 링크 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm

years_urls, years = [], []
years_url = 'https://music.bugs.co.kr/years'

resp = requests.get(years_url)
soup = BeautifulSoup(resp.content, 'lxml')
a_tags = soup.select('a.title.hyrend')

for a_tag in a_tags:

  if a_tag['title'].split('년')[1][0] == '대':
    continue

  years_urls.append(a_tag['href'])
  years.append(a_tag['title'].split('년')[0])


* 각 링크 내 곡목록 가져오기

In [ ]:
lyrics_url, title, artist, year = [], [], [], []

for i in tqdm(range(len(years_urls))):
  url = years_urls[i]
  resp = requests.get(url)
  soup = BeautifulSoup(resp.content, 'lxml')

  lyrics_urls = soup.select('table.trackList td a.trackInfo')
  titles = soup.select('table.trackList th p.title')
  artists = soup.select('section.sectionPadding.contents table.list.trackList td.left p.artist a')

  # 여러 가수가 불렀을 경우 첫번째 가수만 가져오도록 처리.
  for artist_tag in artists:
    if artist_tag['title'] == '아티스트 전체보기':
      continue
    artist.append(artist_tag)

  # 금지곡인 경우 a 태그 말고 span 태그이기에 따로 처리.
  for title_tag in titles:
    
    title_a_tags = title_tag.select('a')
    title_span_tags = title_tag.select('span')

    if len(title_a_tags) > 0:
      title.append(title_a_tags[0]['title'])
    else:
      title.append(title_span_tags[1].text)


  lyrics_url.extend(lyrics_urls)
  year.extend([years[i]]*len(lyrics_urls))

  # if len(lyrics_url) != len(title):
  #   print(i, ':',  url)

print(len(lyrics_url), len(title), len(artist), len(year))


* 각 링크에서 추출한 가사 링크 접속해서 가사 추출하기

In [ ]:

lyrics = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
    ,'Referer': 'https://music.bugs.co.kr/track/6065263?wl_ref=list_tr_08_mab'
}

cookie = {'cookie':
          '_ga=GA1.3.1816123160.1667278014; _gid=GA1.3.1405268211.1667278014; _gac_UA-134713316-19=1.1667278014.CjwKCAjw5P2aBhAlEiwAAdY7dEGLFCONQRkEoA0kvJenxh8wG6mMv1kbFZ0E2aS9CghY6E4Whd4fORoC6yEQAvD_BwE; _gac_UA-97174709-1=1.1667278014.CjwKCAjw5P2aBhAlEiwAAdY7dEGLFCONQRkEoA0kvJenxh8wG6mMv1kbFZ0E2aS9CghY6E4Whd4fORoC6yEQAvD_BwE; cto_bundle=8l24E19TTzRyYSUyQmxYZUhmbEoyWTRZMSUyRkppS2hWVTVhWmpmV3YlMkJpZSUyRkwlMkYxY001OXprT1dRRUdYSHdiTDhsOTJjdHFOU1JuRnpqU21TczNkemwzcyUyQjVLVG1LeTdRREpkbUxibXFZczA2OTVoTDJUM2xDR20lMkZpWmE3emN5dXJDeXg2MlM0OUc1WCUyRlRIVyUyQkxLelUxbUJINGJlN1B6SHdaYmV1cjBpRVhNTGlPbWxYdE9OdWpQNVNicTRnQk1MUk5SV3V3bnU; SCOUTER=x5r8m0krm2sd5h; PCID=16672780261962967733626; RC_RESOLUTION=1440*900; RC_COLOR=30; ACEUCI2=1; searchLogSessionId=6C1290A2-8B91-9CC0-9130-7E31289B6F98; ConnectionInfo=4ef8dd71f9769907d352320408bd6356%2C1%3A58081741%3Adeepshadow25%40naver.com%3A%40%3A%40ZGVlcHNoYWRvdzI1QG5hdmVyLmNvbTo3NzExMjgyNDI6ZTM1YjQ0NDYyNzE5MDliZjE5YmYzYWZjMjA4Mzk4NzU%2Cupdated%3A1667347249%2Cip%3A1.215.118.68; UserID=deepshadow25%40naver.com; UserSRL=58081741; chk="t1=t1&BugsNo=DGCERg7AtId&BugsId=x967AK1BhMK-5imhuGwVjxbFM68HzNVpl&key=10790173&BugsName=WB-WtSXyvamkJKVEK3k5JsOluZU9RTcRqUZBWqhDMGdV&BugsMemTp=e&t2=t2"; cp=BugsNo%3DDGCERg7AtId%26BugsId%3Dx967AK1BhMK-5imhuGwVjxbFM68HzNVpl%26key%3D10790173%26BugsEncodingName%3D%25EC%259D%25B4%25EC%2584%25B1%25EC%2597%25B0%26BugsName%3D%C0%CC%BC%BA%BF%AC%26vstat%3DY%26ssn%3D1QNx-DqAQBq%26gender%3DM%26loginIp%3D1.215.118.68%26BugsCI%3DzKgmvnLFE9ET5HmhpPr1RiuPZJzNVszFF6BZS95sRMd5HiE6omeLL%2Bl4sLIumuiSouLitOl2uLywy8bqyfQ5VA%3D%3D%26BugsMemTp%3De; AccessToken=qRK8rbKQaVBeiX5C9SHttxndJWofSW3eKu_RfI2ERoBsrBTEmR-rubx; OauC=YTo0OntzOjEyOiJhY2Nlc3NfdG9rZW4iO3M6NTU6InFSSzhyYktRYVZCZWlYNUM5U0h0dHhuZEpXb2ZTVzNlS3VfUmZJMkVSb0JzckJURW1SLXJ1YngiO3M6MTM6InJlZnJlc2hfdG9rZW4iO3M6NTU6IlNfR2V5RThOb0t3RFlweU5JLWd0M3FpdWVjZWYtWFV3ckxORlNHZ2FSVEttVEFtVGJibUs2d3giO3M6ODoic2V0X2RhdGUiO3M6MTM6IjE2NjczNDcyNDk0MzQiO3M6MTA6ImV4cGlyZXNfaW4iO2k6MjUzNDYxNTt9; last_login_log_date_cookie=1667347250942; _ACR0=75ce47f94a08acb60af230e5c61cedb107343795; _lh_time_=1667348090866; wcs_bt=s_595467f7a42:1667348362; _ACU103005=1667278026423717846.1667348362328.1.0.717846MDR9AVRYYYX10.0.0.0.....; _ACS103005=534; JSESSIONID=CBC72D9EC555C97BF75B8114AA8A9E61.s830'}

for i in tqdm(range(len(lyrics_url))): #
  url = lyrics_url[i]['href']
  resp = requests.get(url, headers=headers, cookies=cookie)
  soup = BeautifulSoup(resp.content, 'lxml')

  lyrics_tag = soup.select('div.lyricsContainer xmp')

  if len(lyrics_tag) > 0:
    lyrics.append(str(lyrics_tag[0]).replace('<xmp>', '').replace('</xmp>', '').replace('\r\n', ' '))
  else:
    lyrics.append('NULL')
    print(i, ':', url, 'is NULL')



* 데이터 정제 후 csv 출력

In [ ]:
import pandas as pd


dict_lyrics = {
    'lyrics_url': lyrics_url, 'title': title, 'artist': artist, 'year': year, 'lyrics': lyrics
}

df_lyrics = pd.DataFrame(dict_lyrics)
df_lyrics.to_csv('df_lyrics.csv')

df_lyrics2 = df_lyrics 
for i in range(len(df_lyrics2['artist'])):
  # df_lyrics2['artist'][i] = df_lyrics2['artist'][i].text
  df_lyrics2['lyrics_url'][i] = df_lyrics2['lyrics_url'][i]['href']

import re

for i in range(len(df_lyrics2['lyrics'])):
  if len(re.findall('\t', df_lyrics2['lyrics'][i])) > 0:
    df_lyrics2['lyrics'][i] = df_lyrics2['lyrics'][i].replace('\n', ' ')

df_lyrics2.to_csv('df_lyrics2.csv', encoding='utf-8-sig')

## 지니

* 연도별 곡목록 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm 
first_year = 1970
last_year = 2023

titles, artists, objectIds, years = [], [], [], []
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
    ,'Referer': 'https://www.genie.co.kr/chart/musicHistory?year=1970&category=0&pg=1'
}

for i in tqdm(range(first_year, last_year)):
  
  # page1, 2
  for page_num in range(1, 3):

    url = 'https://www.genie.co.kr/chart/musicHistory?year={0}&category=0&pg={1}'.format(i, page_num)
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.content, 'lxml')

    title_tags = soup.select('td.info a.title')
    artist_tags = soup.select('td.info a.artist')
    objectId_tags = soup.select('td.link a')

    print(url, ' 진행중...')

    if len(title_tags) == 0:
      continue

    # print(len(titles), len(artists), len(objectIds))
    for j in range(len(title_tags)):
      titles.append(title_tags[j].text.replace('\n', '').replace('TITLE', '').strip())
      artists.append(artist_tags[j].text)
      objectIds.append(str(objectId_tags[j]['onclick']).replace("fnViewSongInfo('", "").replace("');return false;", ""))
      years.append(i)

    print(url, ' 완료...')


* 곡목록 가사 가져오기

In [ ]:
import pandas as pd

lyrics = []

cookies = {
    'Cookie': 'JSESSIONID=BF2F040D4696F2E511D38677370E96D9; hazelcast.sessionId=HZ7338C44115814C6DBEB51B214A22839E; _BS_GUUID=jgzjHY0WyrQpiJHxPUouUFiAbuJS1X8leHEWFFSe; genie-player-transok=Y; MusicPlayerCookie=128%3B0%3B0%3B1%3B0.5%3BY%3BY%3BN%3B3%3B1%3BAAC; bomb_banner=18987; _TRK_CR=https%3A%2F%2Fwww.google.com%2F; IS%5FKT%5FPRODUCT%5FFIRST=false; _TRK_UID=f8ee3a3f88692eb619438c8c05cfbcfa:6:0.18005677662037037:1667371129754; _TRK_SID=fc10039f21353f23fd996eacdd92cb3e; _TRK_CQ=%3Fkeywd=6bVET930%26source=adwords%26gclid=CjwKCAjwh4ObBhAzEiwAHzZYU8ohZMS3Nn1z2GvrrnLpNZtpMTHVmYiDscQyGfvGxC-lejYMRpid2xoCEKoQAvD_BwE; GENIE%5FUXM=YfzbtN%2BCC22A3COuoRu1ig%3D%3D; GENIE%5FUXD=iPI7jR85CSz%2BsgqmieUdqA%3D%3D; GENIE%5FUXS=SQf1BDscDXVNDfQ6O56fkA%3D%3D; GENIE%5FUXA=0h3IgHa3bnQzkkjOQ87%2F9A%3D%3D; GENIE%5FUXTK=HKOViWbDObZhGLenOVRYIm%2F9ou2X3ePip6GbjD%2F7ds0%3D; GENIE%5FUCF=a9Z5QiaQm9uspPHH3XAYmw%3D%3D; GENIE%5FUCR=SQf1BDscDXVNDfQ6O56fkA%3D%3D; GENIE%5FUXN=BREgSwiK3NUYBfiHtY1JKA%3D%3D; GENIE%5FUST=cdgM08h0S%2F0DAJAGhMbn5A%3D%3D; GENIE%5FUSD=cdgM08h0S%2F0DAJAGhMbn5A%3D%3D; GENIE%5FUXAD=t7zxVcV63%2FI7CuBp89Ov6Q%3D%3D; GENIE%5FP%5FYN=Y; GENIE%5FP%5FNUM=0; GENIE%5F4000%5FFLAG=false; _TRK_RK=324596111; genie-recent=%5B%7B%22word%22%3A%22%uBC15%uC7AC%uBC94%20%uBAB8%uB9E4%22%2C%22date%22%3A%2211.02%22%7D%2C%7B%22word%22%3A%22%uBC15%uC7AC%uBC94%20%uBAB8%uB9E4%22%2C%22date%22%3A%2211.02%22%7D%5D; parentUrl=/detail/songInfo%3Fxgnm%3D84091479; _TRK_EX=30; REMOVE_ALERT_COOKIE_FLAG=324596111%7C%7C20221102163235'
}

for i in tqdm(range(len(objectIds))):
  url = 'https://www.genie.co.kr/detail/songInfo?xgnm={0}'.format(objectIds[i])

  resp = requests.get(url, headers=headers, cookies=cookies)
  soup = BeautifulSoup(resp.content, 'lxml')
  lyrics_tags = soup.select('pre#pLyrics p')

  for lyric in lyrics_tags:
    lyrics.append(lyric.text.replace('\n', ' '))

  if len(lyrics_tags) == 0:
    lyrics.append('NULL')

dict_lyrics = {
    'title': titles,
    'artist': artists,
    'year': years,
    'lyrics': lyrics
}


* 데이터 정제 및 출력

In [ ]:
print(len(titles), len(artists), len(years), len(lyrics))


df_lyrics = pd.DataFrame(dict_lyrics)
# df_lyrics
# df_lyrics.to_csv('genie_lyrics.csv', encoding='utf-8-sig')

lyrics2 = lyrics

for i in range(len(lyrics2)):
  lyrics2[i] = lyrics2[i].replace('\r', ' ')

dict_lyrics = {
    'title': titles,
    'artist': artists,
    'year': years,
    'lyrics': lyrics2
}

df_lyrics = pd.DataFrame(dict_lyrics)
# df_lyrics
df_lyrics.to_csv('genie_lyrics.csv', encoding='utf-8-sig')

### 각 웹사이트별 출력 자료 반복없이 합치기

* 함수 생성

In [ ]:
# 가수, 제목, 혹은 가요에 있을 수 있는 빈칸 정리
def remove_space(df):
  for i in range(len(df)):
    df['names'][i] = df['names'][i].strip().replace(' ','')
    df['singer'][i] = df['singer'][i].strip()
    df['lyrics'][i] = df['lyrics'][i].replace('\n', ' ')

In [ ]:
# 안 겹치는 노래들의 index 찾기
def find_idx(df1, df2):
  idx_list = []
  for i in range(len(df2['names'])):
    if df2['names'][i] not in list(df1['names']):
      idx_list.append(i)
  return idx_list

In [ ]:
# 3개의 data 합치기
def combine(df1, df2, df3):
  new_year = list(df1['year'])
  new_singer = list(df1['singer'])
  new_names = list(df1['names'])
  new_lyrics = list(df1['lyrics'])
  for num in find_idx(df1, df2):
    new_year.append(df2['year'][num])
    new_singer.append(df2['singer'][num])
    new_names.append(df2['names'][num])
    new_lyrics.append(df2['lyrics'][num])
  for num in find_idx(df1, df3):
    new_year.append(df3['year'][num])
    new_singer.append(df3['singer'][num])
    new_names.append(df3['names'][num])
    new_lyrics.append(df3['lyrics'][num])
  return new_year, new_singer, new_names, new_lyrics

* CSV 파일 불러와서, 하나의 파일로 합쳐주기

In [ ]:
# 3개의 csv 불러오기
df_melon = pd.read_csv('melon_lyrics.csv', encoding = 'cp949')
df_bugs = pd.read_csv('bugs_lyrics.csv')
df_genie = pd.read_csv('bugs_lyrics.csv')

In [ ]:
# 빈 칸 제거
remove_space(df_melon)
remove_space(df_bugs)
remove_space(df_genie)

In [ ]:
# 하나의 csv로 만들어주기
year, singer, names, lyrics = combine(df_melon, df_bugs, df_genie)

dict_songs ={'year':year,
             'singer':singer,
             'names':names,
             'lyrics':lyrics}

df_final = pd.DataFrame(dict_songs)
df_final.to_csv('top_songs_final.csv', encoding = 'utf-8')
df_final

# 전처리

## 구글 번역


In [ ]:
!pip install googletrans==3.1.0a0 

from googletrans import Translator 

# test
translator = Translator()           
translations = translator.translate(['The quick brown fox', '뛰다 over', 'the lazy dog'], dest='ko')
for translation in translations:
  print(translation.origin, ' -> ', translation.text)

In [ ]:
import re
import pandas as pd

translator = Translator() 
# 파일 읽기
df_lyrics = pd.read_excel('song_lyrics.xlsx', index_col=None)

for i in range(len(df_lyrics)): # for lyrics in df_lyrics
  print(i, df_lyrics['lyrics'].loc[i])
  # 가사에서 영어만 추출
  lyrics_list = re.findall('[A-Za-z]+', df_lyrics['lyrics'].loc[i])

  if len(lyrics_list) > 0:
    # 번역
    trnaslated_list = translator.translate(lyrics_list, dest='ko')
    lyrics_without_english = re.sub('[A-Za-z]+', '', df_lyrics['lyrics'].loc[i])
    # 다시 이어붙이기
    df_lyrics['lyrics'].loc[i] = "{0} {1}".format(lyrics_without_english, " ".join([translation.text for translation in trnaslated_list])) 
    print(i, df_lyrics['lyrics'].loc[i])



In [ ]:
df_lyrics.to_excel('song_lyrics_translated.xlsx')

## 라벨링

라벨링한 것을 코드로 읽을 수 있는 형태로 변환.

처리할 수 있는 방법 2가지

* LabelEncoder 사용(Python library)
* Dict(or list) 로 바꿔서 있을 경우 해당 컬럼에 1(True)로 처리. ( ✅ )

In [ ]:
# label encoder 사용
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd

# dict or list 사용
# 희망, 돈, 사랑, 효, 우정, 남녀평등, 욕설, 자연, 후회, 추억, SF, 음식, 자신감, 즐거움
categories = ['희망', '돈', '사랑', '효', '우정', '남녀평등', '욕설', '자연', '후회', '추억', 'SF', '음식', '자신감', '즐거움']

df_lyrics = pd.read_excel('song_lyrics_translated.xlsx', index_col=None)

# add category columns to dataframe
# df_lyrics.loc[:, ['hope', 'money', 'love']] = [0]*len(df_lyrics)

df_lyrics = df_lyrics.assign(hope=[0]*len(df_lyrics)
                            , money=[0]*len(df_lyrics)
                            , love=[0]*len(df_lyrics)
                            , filial=[0]*len(df_lyrics)
                            , friend=[0]*len(df_lyrics)
                            , equal=[0]*len(df_lyrics)
                            , swear=[0]*len(df_lyrics)
                            , nature=[0]*len(df_lyrics)
                            , regret=[0]*len(df_lyrics)
                            , memory=[0]*len(df_lyrics)
                            , sf=[0]*len(df_lyrics)
                            , food=[0]*len(df_lyrics)
                            , confi=[0]*len(df_lyrics)
                            , enjoy=[0]*len(df_lyrics))
df_lyrics

In [ ]:
# 희망, 돈, 사랑, 효, 우정, 남녀평등, 욕설, 자연, 후회, 추억, SF, 음식, 자신감, 즐거움
from tqdm import tqdm

for index in tqdm(range(len(df_lyrics))):
  if "희망" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['hope'].loc[index] = 1
  if "돈" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['money'].loc[index] = 1
  if "사랑" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['love'].loc[index] = 1
  if "효" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['filial'].loc[index] = 1
  if "우정" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['friend'].loc[index] = 1
  if "남녀평등" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['equal'].loc[index] = 1
  if "욕설" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['swear'].loc[index] = 1
  if "자연" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['nature'].loc[index] = 1
  if "후회" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['regret'].loc[index] = 1
  if "추억" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['memory'].loc[index] = 1
  if "SF" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['sf'].loc[index] = 1
  if "sf" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['sf'].loc[index] = 1
  if "음식" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['food'].loc[index] = 1
  if "자신감" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['confi'].loc[index] = 1
  if "즐거움" in df_lyrics['categories'].loc[index].replace(' ', '').split(','): 
    df_lyrics['enjoy'].loc[index] = 1

  0%|          | 0/6200 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 6200/6200 [00:05<00:00, 1239.63it/s]


In [ ]:
df_lyrics.to_excel('song_lyrics_labeled.xlsx')

In [ ]:
df_categories_sum.sort_values(["hope"], ascending=[False]).head(10)

## 빈도수 확인

불용어를 빈도 수 많은 토큰을 우선으로 살펴본 후 선정하기 위해.

In [ ]:
# 크롤링한 모든 노래에 대하여, 특정 단어의 출현 빈도수 
def find_frequency(df):

  lst = ['Noun', 'Adjective', 'Adverb', 'Verb']
  final = []
  
  for i in tqdm(range(df.shape[0])):
    test_txt = re.sub("[^가-힣\\s]", "", df['lyrics'].iloc[i])
    test_review = okt.pos(test_txt, stem=True)
    for j in range(len(test_review)):
      if test_review[j][1] in lst:
        final.append(test_review[j][0])

  num_counts = Counter(final)
  sorted_counts = sorted(num_counts.items(), key=lambda x:x[1])

  return sorted_counts

In [ ]:
# count는 각 단어별로 개수
count = find_frequency(songs)

- 단어마다 문서 출현 빈도수 측정

In [ ]:
dic_list = []
# 노래 가사에 나온 단어들만 추출해낸 리스트 생성
for i in range(len(count)):
  dic_list.append(count[i][0])

dic_list

In [ ]:
# 각 단어를 key 로 설정하고 value 는 모두 0으로 지정
dictionary = dict.fromkeys(dic_list, 0)

dictionary

In [ ]:
# 노래별 가사를 토큰화시킨 리스트를 이중 리스트로 작성
def clean_lyrics(df):

  lst = ['Noun', 'Adjective', 'Adverb', 'Verb']
  final = []
  
  for i in tqdm(range(df.shape[0])):
    result = []
    test_txt = re.sub("[^가-힣\\s]", "", df['lyrics'].iloc[i])
    test_review = okt.pos(test_txt, stem=True)
    for j in range(len(test_review)):
      if test_review[j][1] in lst:
        result.append(test_review[j][0])
    final.append(result)
  
  return final

In [ ]:
# 가사 전처리 및 토큰화
lyrics_list = clean_lyrics(songs)

In [ ]:
# 문서 빈도수 측정
def doc_frequency(lyrics, dictionary):

  for lyric in tqdm(lyrics):
    num_count = Counter(lyric)
    for word in dic_list:
      if num_count[word] > 0:
        dictionary[word] += 1
  return dictionary

In [ ]:
doc_frequency(lyrics_list, dictionary)

- 리턴된 데이터에서 df 확인 후, df 상위 2% 데이터를 직접 검토하여 불용어 사전 작성

In [ ]:
## 16000개 가량의 단어중에 상위 2% 문서 출현 빈도 단어들 중, 불용어로 지정할 단어들만 모은 리스트를 생성
stop_list = ['잠', '바', '워', '몸', '진짜', '언젠가', '죽다', '힘', '뭐라다', '절대', '그저', '애', '하얗다', '돌다', '누가', '크다', '늘', '다시다', '뜨겁다', '아침', '가지', '숨', '상처', '뛰다', '순', '듣다', '머리', '단지', '이상', '꼭', '어리다', '기다', '따르다', '채', '향', '떨어지다', '듯', '예', '뜨다', '아직도', '갖다', '전', '나오다']
stop_list2 = ['받다', '때문', '지다', '서다', '이름', '많이', '수가', '조금', '아래', '두', '좀', '의', '무엇', '한번']
stop_list3 = ['줄', '처럼', '누구', '뿐', '만들다', '게', '순간', '어디', '미치다', '다른', '길', '뭐', '음', '안녕', '저', '건', '하루', '그래서', '여기', '이렇다', '손', '그렇게','우', '자다', '많다', '노래', '뒤', '응', '곳', '말다', '거', '정말', '대다', '이다', '웃다', '못', '하늘', '에스', '어떻다', '없이', '해주다', '곁', '나다', '티', '모든', '이렇게', '들다']
stop_list4 = ['하다', '있다', '없다', '말', '되다', '그렇다', '생각', '걸', '그', '것', '오', '이', '다', '속','해',
 '돼다', '알', '젠', '나르다', '주다', '아기', '중', '사람', '오다','가다','않다']

In [ ]:
stop_list.extend(stop_list2)
stop_list.extend(stop_list3)
stop_list.extend(stop_list4)
print(stop_list)

In [ ]:
#불용어 사전을 이용하여 최종 전처리 작업
def preprocessing(review): 
    okt = Okt()
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣\\s]", "", review)
    
    # 2. okt 객체를 활용해서 형태소 토큰화 + 품사 태깅
    word_review = okt.pos(review_text, stem=True)
    
    # 노이즈 & 불용어 제거
    word_review = [(token, pos) for token, pos in word_review if not token in stop_list]
    
    # 명사, 동사, 형용사 추출
    word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective', 'Adverb']]

    return word_review

- 토큰화하여, X_train과 X_test tfidf 형태로 생성

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=preprocessing, max_features=3000, min_df=5, max_df=0.5) 

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.toarray()

# GridSearchCV

##모델링

- 모델 생성

In [ ]:
# 분류모델을 담을 리스트 생성 
classifiers = []

# random_state 설정
random_state = 2

# 데이터프레임 생성을 위하여 알고리즘 제목 저장
clf_names = ['Decision Tree','Random Forest', 'AdaBoost', 'Extra Trees', 'Gradient Boost',
             'MLP', 'KNN', 'Support Vector', 'Logistic Regression', 'Linear Discrimination']

# 트리 기반 알고리즘
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))

# 그 외 선형, 신경망 등의 알고리즘
classifiers.append(KNeighborsClassifier())
classifiers.append(SVC(random_state=random_state))
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(LinearDiscriminantAnalysis())

In [ ]:
# 레이블인코딩된 데이터로 교차검증을 수행하여 리스트에 저장

kfold = StratifiedKFold(n_splits=10)

cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier, X_train_tfidf, y = y_train,
                                      scoring = "accuracy", cv = kfold, n_jobs=4))

In [ ]:
# 교차검증 점수 리스트 출력
cv_results

In [ ]:
# 모델들의 교차검증 점수의 평균과 표준편차를 계산하여 리스트에 저장
cv_means = []
cv_std = []

for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

In [ ]:
# 데이터프레임으로 변환
results_le = pd.DataFrame(cv_results, index=clf_names)
results_le

In [ ]:
# 모델별 교차검증 점수 평균, 표준편차를 새로운 칼럼으로 추가
results_le['mean'] = cv_means
results_le['std'] = cv_std

In [ ]:
#  모델별 교차검증 점수 평균 시각화
g = sns.barplot("mean",results_le.index ,data = results_le, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")

- kfold 를 10 으로 설정시에 다양한 모델을 돌리기에 다소 무리가 있었으므로, 데이터가 편향도를 확인 후, 그렇지 않을 시에 kfold를 낮추기로 결정

In [ ]:
# 평균으로부터 2 표준편차 이상 벗어난 데이터 포인트가 데이터의 10%를 넘지 않을 시, kfold update
def find_outliers(df):
  lst = []
  rows = df.shape[0]-1
  cols = df.shape[1]-2
  for i in range(0,rows):
    for j in range(0,cols):
      if df.iloc[i][j] > df.iloc[i]['mean'] + 2*df.iloc[i]['std'] or df.iloc[i][j] < df.iloc[i]['mean'] - 2*df.iloc[i]['std']:
        lst.append((i,j))
  if len(lst) < 0.1 * (rows * cols):
    print('Update Kfold')
  else:
    print('Do not change Kfold')

In [ ]:
find_outliers(results_le)

In [ ]:
kfold = StratifiedKFold(n_splits=3)

## 하이퍼 파라미터 튜닝

### Random Forest

In [ ]:
# Random Forest 객체 생성
RFC = RandomForestClassifier(random_state=7)


# param_grid 설정
rf_param_grid = {"max_depth": [None],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}

# Grid Search 객체 생성
gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsRFC.fit(X_train_tfidf,y_train)

In [ ]:
# 최적의 파라미터 조합 출력
RFC_best = gsRFC.best_estimator_
RFC_params = gsRFC.best_params_
print(RFC_best)
print(RFC_params)

# 최고 점수(accuracy) 출력
print(gsRFC.best_score_)

### KNN (K-Nearest Neighbors)

In [ ]:
# KNN 객체 생성
KNN = KNeighborsClassifier()

# param_grid 설정
knn_param_grid = {"n_neighbors" : [1, 2, 3, 4, 5], 
                  "algorithm": ['auto'],
                  "weights" : ['uniform', 'distance'],
                  "leaf_size" : [1, 2, 3, 4, 5, 10]}

# Grid Search 객체 생성
gsKNN = GridSearchCV(KNN, param_grid = knn_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsKNN.fit(X_train_tfidf, y_train)

In [ ]:
# 최적의 파라미터 조합 출력
KNN_best = gsKNN.best_estimator_
KNN_params = gsKNN.best_params_
print(KNN_best)
print(KNN_params)
# 최고 점수(accuracy) 출력
print(gsKNN.best_score_)

### AdaBoost

In [ ]:
# AdaBoost 객체 생성
ADA = AdaBoostClassifier(random_state=7)

# param_grid 설정
ada_param_grid = {"n_estimators" :[1,20,30],
                  "algorithm" : ["SAMME","SAMME.R"],                  
              "learning_rate":  [0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

# Grid Search 객체 생성
gsADA = GridSearchCV(ADA, param_grid = ada_param_grid, cv=kfold, scoring="accuracy", verbose = 1)

# Grid Search 실행
gsADA.fit(X_train_tfidf, y_train)

In [ ]:
# 최적의 파라미터 조합 출력
ADA_best = gsADA.best_estimator_
ADA_params = gsADA.best_params_
print(ADA_best)
print(ADA_params)

# 최고 점수(accuracy) 출력
print(gsADA.best_score_)


### Gradient Boost

In [ ]:
# Gradient boost 객체 생성
GBC = GradientBoostingClassifier(random_state=7)

# param_grid 설정
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              }
# Grid Search 객체 생성
gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsGBC.fit(X_train_tfidf,y_train)

In [ ]:
# 최적의 파라미터 조합 출력
GBC_best = gsGBC.best_estimator_
GBC_params = gsGBC.best_params_
print(GBC_best)
print(GBC_params)

# 최고 점수(accuracy) 출력
print(gsGBC.best_score_)

### Extra Trees

In [ ]:
#ExtraTrees 객체 생성
Ext = ExtraTreesClassifier(random_state=7)


# param_grid 설정
ex_param_grid = {"max_depth": [None],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}

# Grad Search 객체 생성
gsExt = GridSearchCV(Ext,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsExt.fit(X_train_tfidf,y_train)

In [ ]:
# 최적의 파라미터 조합 출력
Ext_best = gsExt.best_estimator_
Ext_params = gsExt.best_params_
print(Ext_best)
print(Ext_params)

# 최고 점수(accuracy) 출력
print(gsExt.best_score_)

### Decision Tree

In [ ]:
# Decision Tree 객체 생성
DTC = DecisionTreeClassifier(random_state=7, )

# param_grid 설정
dtc_param_grid = {"criterion" : ["gini", "entropy"],
              "min_samples_split" :   [2, 4, 6, 10],
              "min_samples_leaf" : [1,3, 5, 10]}

# Grad Search 객체 생성
gsDTC = GridSearchCV(DTC,param_grid = dtc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsDTC.fit(X_train_tfidf,y_train)

In [ ]:
# 최적의 파라미터 조합 출력
DTC_best = gsDTC.best_estimator_
DTC_params = gsDTC.best_params_
print(DTC_best)
print(DTC_params)

# 최고 점수(accuracy) 출력
print(gsDTC.best_score_)

### Support Vector

In [ ]:
#Support Vector 객체 생성
SVM = SVC(random_state=7, probability=True)

# param_grid 설정
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}

# Gradient boost 객체 생성
gsSVM = GridSearchCV(SVM,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsSVM.fit(X_train_tfidf,y_train)

In [ ]:
# 최적의 파라미터 조합 출력
SVM_best = gsSVM.best_estimator_
SVM_params = gsSVM.best_params_
print(SVM_best)
print(SVM_params)

# 최고 점수(accuracy) 출력
print(gsSVM.best_score_)

### MLP(Multi-layer Perceptron)

In [ ]:
# Linear Discrimination Analysis 객체 생성
MLP = MLPClassifier()

# param_grid 설정
mlp_param_grid = {'hidden_layer_sizes': [100, 150, 200, 250], 
                #   'activation': ['identity', 'logistic', 'tanh', 'relu'],
                #   'solver': ['lbfgs','sgd','adam'],
                  'alpha': [ 0.0001, 0.001, 0.01],
                #   'learning_rate': ['const  ant','invscaling','adaptive']
                }

# Gradient boost 객체 생성
gsMLP = GridSearchCV(MLP,param_grid = mlp_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsMLP.fit(X_train_tfidf,y_train)

In [ ]:
# 최적의 파라미터 조합 출력
MLP_best = gsMLP.best_estimator_
MLP_params = gsMLP.best_params_
print(MLP_best)
print(MLP_params)

# 최고 점수(accuracy) 출력
print(gsMLP.best_score_)


### Logistic Regression

In [ ]:
# Logistic Regression 객체 생성
LGS = LogisticRegression(random_state=7)

# param_grid 설정
lgs_param_grid = {'penalty': ['l1','l2'], 
                  'C': np.logspace(0,4,10)}

# Gradient boost 객체 생성
gsLGS = GridSearchCV(LGS,param_grid = lgs_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

# Grid Search 실행
gsLGS.fit(X_train_tfidf,y_train)

In [ ]:
# 최적의 파라미터 조합 출력
LGS_best = gsLGS.best_estimator_
LGS_params = gsLGS.best_params_
print(LGS_best)
print(LGS_params)

# 최고 점수(accuracy) 출력
print(gsLGS.best_score_)

### Voting

In [ ]:
#튜닝한 파라미터로 하드보팅한 후 점수를 출력해보자.
grid_hard = VotingClassifier(estimators = [('Decision Tree', DTC_best), 
                                           ('Random Forest', RFC_best),
                                           ('ExtraTrees', Ext_best),
                                           ('SVC', SVM_best),
                                           ('AdaBoost', ADA_best),
                                           ('GradientBoosting', GBC_best),
                                           ('KNN', KNN_best),
                                           ('MLP', MLP_best),
                                           ('Logistic Regression', LGS_best)], voting = 'hard')

grid_hard_cv = model_selection.cross_validate(grid_hard, X_train_tfidf, y_train, cv=10)

print("Hard voting on test set score mean: {:.2f}". format(grid_hard_cv['test_score'].mean() * 100))

In [ ]:
#튜닝한 파라미터로 하드보팅
grid_soft = VotingClassifier(estimators = [('Decision Tree', DTC_best), 
                                           ('Random Forest', RFC_best),
                                           ('ExtraTrees', Ext_best),
                                           ('SVC', SVM_best),
                                           ('AdaBoost', ADA_best),
                                           ('GradientBoosting', GBC_best),
                                           ('KNN', KNN_best),
                                           ('MLP', MLP_best),
                                           ('Logistic Regression', LGS_best)], voting = 'soft')

grid_soft_cv = model_selection.cross_validate(grid_soft, X_train_tfidf, y_train, cv=10)

print("Soft voting on test set score mean: {:.2f}". format(grid_soft_cv['test_score'].mean() * 100))

### Learning Curve

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """Generate a simple plot of the test and training learning curve"""
    # 그래프 설정
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")

    # learning curve
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    
    # 학습 데이터에 대한 점수의 평균
    train_scores_mean = np.mean(train_scores, axis=1)
    # 학습 데이터에 대한 점수의 표준편차
    train_scores_std = np.std(train_scores, axis=1)
    # 검증 데이터에 대한 점수의 평균
    test_scores_mean = np.mean(test_scores, axis=1)
    # 검증 데이터에 대한 점수의 표준편차
    test_scores_std = np.std(test_scores, axis=1)
    
    # 그래프에 눈금 표시
    plt.grid()

    # fill_between() = 그래프의 두 수평 방향의 곡선 사이를 색상으로 채워서 강조
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    
    # 데이터를 연결하는 선을 plot
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    # 범례 표시
    plt.legend(loc="best")
    return plt

In [ ]:
g = plot_learning_curve(DTC_best,"Decision Tree learning curves",X_train_tfidf,y_train,cv=kfold)
g = plot_learning_curve(RFC_best,"Random Forest learning curves",X_train_tfidf,y_train,cv=kfold)
g = plot_learning_curve(Ext_best,"ExtraTrees learning curves",X_train_tfidf,y_train,cv=kfold)
g = plot_learning_curve(SVM_best,"SVC learning curves",X_train_tfidf,y_train,cv=kfold)
g = plot_learning_curve(ADA_best,"AdaBoost learning curves",X_train_tfidf,y_train,cv=kfold)
g = plot_learning_curve(GBC_best,"GradientBoosting learning curves",X_train_tfidf,y_train,cv=kfold)
g = plot_learning_curve(KNN_best,"KNN learning curves",X_train_tfidf,y_train,cv=kfold)
g = plot_learning_curve(MLP_best,"MLP learning curves",X_train_tfidf,y_train,cv=kfold)
g = plot_learning_curve(LGS_best,"Logistic Regression learning curves",X_train_tfidf,y_train,cv=kfold)

# Multi Labeling

- multi labeling 작업을 위한 array 생성

In [ ]:
def combine_df(df):
  full = []
  for i in tqdm(range(df.shape[0])):
    sem = []
    for j in range(8,22):
      sem.append(songs.iloc[i][j])
    full.append(sem)
  return full

In [ ]:
full = combine_df(songs)
songs['full'] = full

In [ ]:
X = songs.lyrics
y = songs.full

In [ ]:
y = np.array(y.tolist())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)

In [ ]:
tfidf = TfidfVectorizer(tokenizer=preprocessing, max_features=3000, min_df=5, max_df=0.5) 

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.toarray()

## 모델별 결과 도출

### 1. Random Froest 

In [ ]:
RFC = MultiOutputClassifier(estimator= RandomForestClassifier()).fit(X_train_tfidf, y_train)
RFC.predict(X_test_tfidf)

In [ ]:
RFC_score = RFC.score(X_test_tfidf, y_test)

### 2. KNN (K-Nearest Neighbors)

In [ ]:
KNN = MultiOutputClassifier(estimator= KNeighborsClassifier()).fit(X_train_tfidf, y_train)
KNN.predict(X_test_tfidf)

In [ ]:
KNN_score = KNN.score(X_test_tfidf, y_test)

### 3. Ada boost

In [ ]:
ADA = MultiOutputClassifier(estimator= AdaBoostClassifier()).fit(X_train_tfidf, y_train)
ADA.predict(X_test_tfidf)

In [ ]:
ADA_score = ADA.score(X_test_tfidf, y_test)

### 4. Gradient Boost

In [ ]:
GRD = MultiOutputClassifier(estimator= GradientBoostingClassifier()).fit(X_train_tfidf, y_train)
GRD.predict(X_test_tfidf)

In [ ]:
GRD_score = GRD.score(X_test_tfidf, y_test)

### 5. Extra Trees

In [ ]:
EXT = MultiOutputClassifier(estimator= ExtraTreesClassifier()).fit(X_train_tfidf, y_train)
EXT.predict(X_test_tfidf)

In [ ]:
EXT_score = EXT.score(X_test_tfidf, y_test)

### 6. Decision Tree

In [ ]:
DTC = MultiOutputClassifier(estimator= DecisionTreeClassifier()).fit(X_train_tfidf, y_train)
DTC.predict(X_test_tfidf)

In [ ]:
DTC_score = DTC.score(X_test_tfidf, y_test)

### 7. SVC

In [ ]:
SVM = MultiOutputClassifier(estimator= SVC()).fit(X_train_tfidf, y_t)
SVM.predict(X_test_tfidf)

In [ ]:
SVM_score = SVM.score(X_test_tfidf, y_test)

### 8. MLP(Multi-layer Perceptron)

In [ ]:
MLP = MultiOutputClassifier(estimator= MLPClassifier()).fit(X_train_tfidf, y_train)
MLP.predict(X_test_tfidf)

In [ ]:
MLP_score = MLP.score(X_test_tfidf, y_test)

### 9. Logistic Regression

In [ ]:
LGS = MultiOutputClassifier(estimator= LogisticRegression()).fit(X_train_tfidf, y_train)
LGS.predict(X_test_tfidf)

In [ ]:
LGS_score = LGS.score(X_test_tfidf, y_test)

### Bar graph

In [ ]:
multi_dict = {'RFC':RFC_score,
              'KNN':KNN_score,
              'ADA':ADA_score,
              'GRD':GRD_score,
              'EXT':EXT_score,
              'DTC':DTC_score,
              'SVM':SVM_score,
              'MLP':MLP_score,
              'LGS':LGS_score}

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(len(multi_dict)), list(multi_dict.values()), align='center')
plt.xticks(range(len(multi_dict)), list(multi_dict.keys()))
plt.show()

In [ ]:
print(RFC_score, KNN_score, ADA_score, GRD_score, EXT_score, DTC_score, SVM_score, MLP_score, LGS_score)